We want to count the number of oil tanks in Houston TX, a major global oil storage facility.  
We'll look for images over the area of interest, download a small segment that contains oil tanks,
and run some feature extraction experiments using PROTOGEN. 
When we are happy with our code, we'll deploy it as a GBDX task on the entirety of the area of interest.

Create a GBDX interface using [gbdxtools](https://github.com/digitalglobe/gbdxtools). You need your credentials to do this; you can find them under your profile on gbdx.geobigdata.io.

In [ ]:
import os
os.environ['GBDX_USERNAME'] = 
os.environ['GBDX_PASSWORD'] = 
os.environ['GBDX_CLIENT_ID'] =  
os.environ['GBDX_CLIENT_SECRET'] = 

import gbdxtools
gbdx = gbdxtools.Interface()

Use one of DigitalGlobe's material selection sites to explore the catalog.

In [ ]:
from IPython.display import IFrame
IFrame('https://discover.digitalglobe.com', width=1600, height=800)

You can also use gdbxtools (but it is less visually appealing ;)

In [ ]:
# Find WorldView-3 PAN+MS imagery with low cloud cover
filters = ["sensorPlatformName = 'WORLDVIEW03'",
           "cloudCover < 10",
           "imageBands = 'Pan_MS1_MS2'"]

# Pick an address
address = 'Houston, TX'
results = gbdx.catalog.search_address(address=address,
                              startDate="2016-01-01T00:00:00.000Z",
                              endDate="2016-11-01T00:00:00.000Z",
                              filters=filters)

# Results is a dictionary with multiple entries. Each entry corresponds to an image over the area of interest.
for result in results:
    properties = result['properties']
    print 'catid: ' + properties['catalogID']
    print 'cloud cover: ' + properties['cloudCover']
    print 'off nadir: ' + properties['offNadirAngle']

Pick an image; the lowest the cloud cover and the off nadir angle, the better the quality of the results.
We've selected 104001001838A000 (WV3, 2016).

The following batch GBDX workflow orders the raw image from the DG factory, and then produces an orthorectified panchromatic image via the AOP_Strip_Processor. Once the workflow completes, the image is stored under platform-stories/oil-tanks/image-houston-pan. This is a time-consuming step which you can skip; we've run this workflow for you.

In [ ]:
catid = '104001001838A000'

# create order task
# it the images are not on GBDX, this task will order them from the DG factory
order = gbdx.Task('Auto_Ordering')
order.inputs.cat_id = catid
# for this particular task, we need to set this flag to true
order.impersonation_allowed = True

# panchromatic
aop_pan = gbdx.Task('AOP_Strip_Processor')
aop_pan.inputs.data = order.outputs.s3_location.value
aop_pan.inputs.bands = 'PAN'
aop_pan.inputs.enable_acomp = False
aop_pan.inputs.enable_pansharpen = False
aop_pan.inputs.enable_dra = False
aop_pan.inputs.ortho_epsg='UTM'

# define preprocessing workflow
preprocess_wf = gbdx.Workflow([order, aop_pan])

# set output location 
output_location = 'platform-stories/oil-tanks/image-houston'
preprocess_wf.savedata(aop_pan.outputs.data, output_location + '-pan')

# execute
preprocess_wf.execute()
preprocess_wf.status

To inspect this image in full resolution, we'll create a leaflet map using the IDAHO TMS service (http://gbdxdocs.digitalglobe.com/docs/idaho-course) and ipyleaflet. 
We can draw a polygon on the map over a region of interest and get its bounding box.

In [ ]:
from ipyleaflet import Map, TileLayer, DrawControl
from shapely.geometry import shape
import sys

catid = '104001001838A000'

# get tms url and bounding box for each idaho image corresponding to this catid
urls, bboxes = gbdx.idaho.get_tms_layers(catid)

# center the map based on idaho image bounds
center = [sum(x)/len(x) for x in zip(*[((N+S)/2.0, (W+E)/2.0) for (W,S,E,N) in bboxes])]

m = Map(center=center, zoom=10)

# add idaho images
for url in urls:
    m.add_layer(TileLayer(url=url))

# enable rectangle draw
dc = DrawControl(polygon={'shapeOptions': {'color': '#00f5FF'}}, polyline={})
def handle_draw(self, action, geo_json):
    geom = shape(geo_json['geometry'])
    print 'W, S, E, N = %s\n' % (str(geom.bounds))    
dc.on_draw(handle_draw)
m.add_control(dc)
    
# launch map    
m

Once we have the bounding box, we can use IDAHO to retrieve the corresponding image chip.

In [ ]:
W, S, E, N = (-95.06904982030392, 29.7187207124839, -95.06123922765255, 29.723901202069023)
chip_geo = 'houston_geo.tif'
gbdx.idaho.get_chip(coordinates=[W, S, E, N], catid = catid, chip_type='PAN', filename=chip_geo)

The chip is in geographic projection, so it needs to be reprojected to UTM to minimize distortion.

In [ ]:
chip = 'houston_utm.tif'
import utm
easting, northing, utm_number, utm_letter = utm.from_latlon(S, W)
reproject = "gdalwarp -t_srs '+proj=utm +zone={} +datum=WGS84' -overwrite {} {}".format(utm_number, chip_geo, chip)
import subprocess
proc = subprocess.Popen([reproject], shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out, err = proc.communicate()

Check out the plot below. This will convince you why the UTM projection is important when performing morphological operations.

In [ ]:
# Define plotting functions
from matplotlib import pyplot as plt
%matplotlib inline
import gdal

def plot_one(chip):
    plt.figure(figsize=(10,5))
    sample = gdal.Open(chip)
    img = sample.ReadAsArray()
    plt.imshow(img, cmap='Greys_r')
    
def plot_two(chip1, chip2):
    plt.figure(figsize=(20,10))
    plt.subplot(121)
    sample1 = gdal.Open(chip1)
    img1 = sample1.ReadAsArray()
    plt.imshow(img1, cmap='Greys_r')
    plt.subplot(122)
    sample2 = gdal.Open(chip2)
    img2 = sample2.ReadAsArray()
    plt.imshow(img2, cmap='Greys_r')

plot_two(chip_geo, chip)

We'll now use protogen extract module in order to detect the oil tanks.

In [ ]:
import protogen

e = protogen.Interface('extract', 'objects')
e.extract.objects.type = 'tanks'
e.extract.objects.visualization = 'binary'
e.extract.objects.multiplier = 2.0
e.extract.objects.dark_hole_size = 20          # remove holes with size < 20m2
e.extract.objects.dark_line_radius = 1         # remove dark lines with radius < 1m
e.extract.objects.bright_line_radius = 1       # remove bright lines with radius < 1m
e.extract.objects.bright_patch_size = 20       # remove bright patches with size < 20m2
e.image = chip

e.athos.tree_type = 'max_tree'
e.athos.area.usage = ['remove if outside']
e.athos.area.min = [100.0]                     # keep nodes with area between min and max
e.athos.area.max = [3500.0]
e.athos.compactness.usage = ['remove if less'] # keep nodes with compactness greater than min
e.athos.compactness.min = [0.97]
e.athos.compactness.export = [1]

# Execute PROTOGEN with the current configuration 
e.execute()

plot_two(chip, e.output)

The output is a new image containing just the oil tanks. We'll use the protogen vectorizer module to derive a geojson with the bounding boxes of the oil tanks.

In [ ]:
v = protogen.Interface('vectorizer', 'bounding_box')
v.image = e.output
v.vectorizer.bounding_box.filetype = 'geojson'
v.vectorizer.bounding_box.target_bbox = False
v.vectorizer.bounding_box.target_centroid = True
v.vectorizer.bounding_box.processor = True
v.athos.tree_type = 'union_find'                       
v.athos.area.export = [1]
v.execute()

Let's check how many vectors were extracted.

In [ ]:
import json

with open(v.output) as f:
    data = json.load(f)

n_oil_tanks = len(data['features'])
print 'There are ' + str(n_oil_tanks) + ' oil tanks in the area of interest'

We can view the vectors on the slippy map (you might need to scroll around to see them).

In [ ]:
# add vectors
from ipyleaflet import GeoJSON

g = GeoJSON(data=data)

# launch map
m.add_layer(g)

m

We'll now deploy our code on a large rectangle by 'taskifying' the extractor and the vectorizer and running a GBDX workflow.
Taskification is accomplished by passing the protogen object as a pickled string to the protogen-runner task. 

In [ ]:
# Define rectangle
W, S, E, N = (-95.14483630657196, 29.696617936567343, -94.98828113079071, 29.773830057098092)
e.image_config.input_latlong_rectangle = [W, N, E, S]

import pickle

def taskify(p):
    "Create instance of a GBDX task that runs a protogen.Interface() object p"
    t = gbdx.Task('protogen-runner')
    t.inputs.pickle = pickle.dumps(p)
    return t

# Taskify protogen objects
extract = taskify(e)
vectorize = taskify(v)

# Chain tasks: output port 'output' of extract task points to input port 'image' of vectorize task
extract.inputs.image = 's3://gbd-customer-data/32cbab7a-4307-40c8-bb31-e2de32f940c2/platform-stories/oil-tanks/image-houston-pan'
vectorize.inputs.image = extract.outputs.output.value

# Define workflow
wf = gbdx.Workflow([extract, vectorize])

# Save extracted oil tank vectors
import uuid
from os.path import join
random_str = str(uuid.uuid4())
output_location = join('platform-stories/trial-runs', random_str)
print output_location
wf.savedata(extract.outputs.output, output_location)
wf.savedata(vectorize.outputs.output, output_location)

In [ ]:
wf.execute()

In [ ]:
wf.status

Once the workflow is complete, download the results.

In [ ]:
gbdx.s3.download(location=output_location, local_dir=random_str)

... count how many there are...

In [ ]:
import json

# open geojson in directory
with open(join(random_str, [x for x in os.listdir(random_str) if '.geojson' in x][0])) as f:
    data = json.load(f)

n_oil_tanks = len(data['features'])
print 'There are ' + str(n_oil_tanks) + ' oil tanks in the area of interest'

... and add the vector layer on the slippy map

In [ ]:
from ipyleaflet import GeoJSON

g = GeoJSON(data=data)

for feature in data['features']:
    feature['properties']['style'] = {'fillOpacity':0}

# launch map
m.add_layer(g)
m